In [1]:
import os
#It goes back to the previous folder (main folder) as we're in research folder
os.chdir('../')

In [2]:
print(os.getcwd())

d:\Data Science\Python Assignment\End to End Kidney Disease Detection


Update the entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

#returning all config parameters which will be used in arrow function output entities
#@dataclass(frozen=True) is a decorator in Python's dataclasses module that makes instances of the class immutable after creation. 
# When applied, it provides the following benefits:
# Immutability: Once an obSject is created, you cannot change its attributes. This is enforced by raising an error if there is any attempt to modify the instance's fields.
# Hashability: Instances of the class become hashable (i.e., you can use them as keys in dictionaries or add them to sets) as long as all their fields are hashable.
@dataclass(frozen=True)
class TrainingMLConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size:int
    params_image_size:list
    params_is_augmentation: bool


Update the configuration manager in src config

In [4]:
from DiseaseClassifier.constants import *
from DiseaseClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


    
class ConfigurationManager:
    def __init__(self,
                 #from constants
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        #init file for referencing the config and params files
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        #we have artifacts_root in config file
        #It'll go through directories and create artifacts folder and subfolders
        create_directories([self.config.artifacts_root])

    #This method creates the directory structure needed for the base machine learning configuration 
    #and returns an instance of PrepareBaseMLConfig
    #we've defined DataIngestionConfig class above
    def get_training_ML_config(self)->TrainingMLConfig:
        training=self.config.training_ML
        prepare_base_model=self.config.prepare_base_ML
        params=self.params
        #referring to the data which will be split into train and validation
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image-samples")
        #referring to prepare_base_ML, artifacts, config file
        create_directories([training.root_dir])

        training_ml_config=TrainingMLConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_is_augmentation=params.AUGUMENTATION
            )
        return training_ml_config


In [5]:
from DiseaseClassifier.constants import CONFIG_FILE_PATH
print(CONFIG_FILE_PATH)

config\config.yaml


In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

#What kwargs Does:
# It enables you to pass an arbitrary number of keyword arguments (arguments with names) to a function.
# Inside the function, kwargs is treated as a dictionary, where each key corresponds to the argument name and each value corresponds to the argument value.

class Training:
    #look at the parameters
    def __init__(self,config:TrainingMLConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    #This code was copied from the keras documentation
    #This code configures TensorFlow's ImageDataGenerator objects to load and process image data from a directory for both training and validation purposes. 
    #It handles optional data augmentation for the training data and ensures that validation data is loaded without augmentation
    
    def train_valid_generator(self):
        #It sets up data generators for both training and validation datasets using the images from a specified directory.
        #The variable datagenerator_kwargs is used to pass common preprocessing settings to the ImageDataGenerator in your code, 
        #but it was not explicitly defined in the snippet. Typically, this dictionary would contain parameters that control how the images are preprocessed before being fed into the model.
        datagenerator_kwargs = dict(
            #This is used to normalize the pixel values in the images. For instance, rescale=1./255 scales pixel values from [0, 255] to [0, 1], which is common when feeding image data into neural networks.
            rescale = 1./255,
            validation_split=0.20
        )

        #all properties for the dataflow
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            #Chooses how to resize the images
            interpolation="bilinear"
        )
        #The ImageDataGenerator is initialized using datagenerator_kwargs, which is presumably a dictionary containing common preprocessing steps
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        #generating validation data
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        #If augmentation is enabled (self.config.params_is_augmentation is True), a new ImageDataGenerator is created with several data augmentation techniques applied
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                #Randomly shifts images horizontally and vertically by 20%.
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            #it is used again to load the training data from the same directory as the validation set
            train_datagenerator = valid_datagenerator

        #generating training data
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )




    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)


    def train(self):
        #This divides and calculates the number of batches (or "steps") in one epoch. It's computed as the total number of training samples divided by the batch size of self.train_generator.
        self.step_per_epoch=self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.step_per_epoch,
            #Defines how many batches of validation data will be processed after each epoch.
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )
        self.save_model(path=self.config.trained_model_path,model=self.model)

    


    
    

Creating the data pipeline

In [6]:
try:
    config= ConfigurationManager()
    training_config=config.get_training_ML_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e


[2024-10-29 16:13:04,999: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-29 16:13:05,036: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-29 16:13:05,038: INFO: common: created directory at: <class 'pathlib.Path'>]
[2024-10-29 16:13:05,039: INFO: common: created directory at: <class 'pathlib.Path'>]
[2024-10-29 16:13:08,754: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 40 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


ValueError: Unknown variable: <KerasVariable shape=(25088, 2), dtype=float32, path=dense/kernel>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.

In [32]:
tf.keras.backend.clear_session()

In [ ]:
self.full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_classes),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy','recall']
        )